In [ ]:
!pip install --upgrade --user google-cloud-aiplatform>=1.29.0 google-cloud-storage langchain pytube youtube-transcript-api chromadb gradio pydantic
# You may safely ignore dependency warnings below

In [ ]:
# get project ID
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
REGION = ! gcloud compute project-info describe --project $PROJECT_ID --format="value(commonInstanceMetadata.items.google-compute-default-region)"
LOCATION = REGION[0] # <-- Enter assigned region here.
# generate an unique id for this session
from datetime import datetime
UID = datetime.now().strftime("%m%d%H%M")

In [ ]:
# init the vertexai package
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import VertexAI

llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)



In [ ]:
from langchain.embeddings import VertexAIEmbeddings

# Embedding
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH =5

embeddings = VertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

In [ ]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=XX2XpqklUrE", add_video_info=True)
result = loader.load()


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
docs = text_splitter.split_documents(result)
print(f"# of documents = {len(docs)}")


In [ ]:
embedding_list = embeddings.embed_documents([doc.page_content for doc in docs])
print(f"You have {len(embedding_list)} embeddings")
print(f"Here's a sample of one: {embedding_list[0][:3]}...")


In [ ]:
db = Chroma.from_documents(docs, embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 2})


In [ ]:
qa = RetrievalQA.from_chain_type( llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)


In [ ]:
def sm_ask(question, print_results=True):
  video_subset = qa({"query": question})
  context = video_subset
  prompt = f"""
  Answer the following question in a detailed manner, using information from the text below. If the answer is not in the text,say I dont know and do not generate your own response.

  Question:
  {question}
  Text:
  {context}

  Question:
  {question}

  Answer:
  """
  parameters = {
      "temperature": 0.1,
      "max_output_tokens": 256,
      "top_p": 0.8,
      "top_k": 40
  }
  response = llm.predict(prompt, **parameters)
  return {
      "answer": response
  }


In [ ]:
import gradio as gr
def get_response(input_text):
  response = sm_ask(input_text)
  return response

grapp = gr.Interface(fn=get_response, inputs="text", outputs="text")
grapp.launch(share=True)
